### Setup

In [6]:
import numpy as np
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio

In [7]:
path = 'D:/Summer Practicum/1_Data/worst_5_routes_per_airline.csv'
worst_5_df = pd.read_csv(path)

### Helper Functions

In [8]:
# Get color from colormap
cmap = plt.get_cmap('YlOrRd')

# Helper function to get line color to work with the scaled Normalized values
def get_rgba(norm_val):
    r, g, b, _ = cmap(norm_val)
    return f'rgba({r*255:.0f}, {g*255:.0f}, {b*255:.0f}, 0.8)'


# Helper function to add markers to any flight path for tooltip hovering
def interpolate_coords(lat1, lon1, lat2, lon2, steps=5):
    lats = np.linspace(lat1, lat2, steps)
    lons = np.linspace(lon1, lon2, steps)
    return list(lats), list(lons)

In [10]:
# Iterate through each airline
for airline in worst_5_df['Airline_Name'].unique():
    df_airline = worst_5_df[worst_5_df['Airline_Name'] == airline]

    fig = go.Figure()

    for _, row in df_airline.iterrows():
        color = get_rgba(row['norm_scaled'])

        # Add flight route tracing
        fig.add_trace(go.Scattergeo(
            locationmode='USA-states',
            lon=[row['LON1'], row['LON2']],
            lat=[row['LAT1'], row['LAT2']],
            mode='lines',
            line=dict(width=2 + row['impact_score'], color=color),
            opacity=0.8,
            name=row['AIRPORT_PAIR'],
            hoverinfo='text',
            text=(
                f"Route: {row['AIRPORT_PAIR']}<br>"
                f"Avg Delay: {row['avg_delay']:.1f} min<br>"
                f"Severe Delays: {row['num_severe_delays']}<br>"
                f"Impact Score: {row['impact_score']:.2f}<br>"
                f"Flights: {row['num_flights']}"
            )
        ))

        # Add origin airport label
        fig.add_trace(go.Scattergeo(
            locationmode='USA-states',
            lon=[row['LON1']],
            lat=[row['LAT1'] - 0.3],
            mode='text',
            text=[row['AIRPORT1']],
            textfont=dict(size=10, color='gray'),
            showlegend=False,
            hoverinfo='skip'
        ))

        # Add destination airport label
        fig.add_trace(go.Scattergeo(
            locationmode='USA-states',
            lon=[row['LON2']],
            lat=[row['LAT2'] + 0.3],
            mode='text',
            text=[row['AIRPORT2']],
            textfont=dict(size=10, color='gray'),
            showlegend=False,
            hoverinfo='skip'
        ))


        

    fig.update_layout(
        title=dict(
            text=f"5 Highest-Impact Routes — {airline}",
            x=0.5,
            xanchor='center',
            font=dict(size=18)
        ),
        geo=dict(
            scope='north america',
            projection_type='mercator',  # use 'natural earth' or 'mercator'
            showland=True,
            landcolor='rgb(240,240,240)',
            showcountries=True,
            countrycolor='gray',
            lonaxis=dict(range=[-130, -60]),
            lataxis=dict(range=[20, 55])
        ),
        showlegend=False
    )

    # Save the figure as image
    filename = f"D:/Summer Practicum/3_Exports/delayed_routes_maps/{airline}_worst_routes.jpg"
    pio.write_image(fig, filename, format='png', width=1200, height=800)